# **TP3 - Exercicio 1 - Grupo 11**

Bruno Miguel Fernandes Araújo - A97509

Tiago Emanuel Lemos Teixeira - A97666

1. Construa um FOTS  $ \Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$ usando este modelo nos inteiros. 

2. Considere como propriedade de segurança
                        `safety = (r > 0) and (r < N) and (r = a*s + b*t`)
                        
   Prove usando $k$-indução que esta propriedade se verifica em qualquer traço do FOTS
3. Prove usando “Model-Checking” com interpolantes e invariantes prove também que esta propriedade é um invariante em qualquer traço de $\,\Sigma\,$.

## Contrução do fots  $ \Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$

Para a construção do fots usaremos como no trabalho anterior a função **init** e a **trans** mas alteradas de forma a envolver inteiros em vez de bitvectors e de forma a ter uma nova condição sobre **a** e **b** relativamente a **N**. Além destas temos **genState** e **genTrace** (usadas nas aulas) que funcionarão de uma forma parecida com o declare e o geratraço do trabalho anterior.

Ps: No trabalho anterior tinhamos a e b como simbolos mas decidimos remove-los por uma questão de simplicidade.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools 

In [2]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

In [3]:

def init(state):
    #pc == 0 && a > 0 && a<N && b > 0 && b<N && r==a && s == 1 && t == 0
    PC = Equals(state['pc'],Int(0)) 
    A = GT(Int(a),Int(0))
    AN = LT(Int(a),Int(N))
    B = GT(Int(b),Int(0))
    BN = LT(Int(b),Int(N))
    R = Equals(state['r'],Int(a))
    S = Equals(state['s'],Int(1))
    T = Equals(state['t'],Int(0))
    return And(PC,A,AN,B,BN,R,S,T)


In [4]:
def trans(curr, prox):
    
    t01= And(
             Equals(curr['pc'],Int(0)),#pc==0
             Equals(prox['pc'],Int(1)),#pc==1 
             Equals(prox['r'],Int(a)),#r==a
             Equals(prox['r_'],Int(b)),#r'==b
             Equals(prox['s'],Int(1)),#s==1
             Equals(prox['s_'],Int(0)),#s'==0
             Equals(prox['t'],Int(0)),#t==0
             Equals(prox['t_'],Int(1))#t'==1
            )
    
  #  print(t01)
  #  print("\n")
    
    t12= And(Equals(curr['pc'],Int(1)),#pc==1
             Not(Equals(curr['r_'],Int(0))),#r'!=0
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(2))#pc==2
            )
    
  #  print(t12)
  #  print("\n")

    t23= And(Equals(curr['pc'],Int(2)),#pc==2
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(3)),#pc==3
             Equals(prox['q'],Div(curr['r'],curr['r_']))
    )
    
        
  #  print(t23)
  #  print("\n")

    
    t31= And(Equals(curr['pc'],Int(3)),#pc==3
             Equals(prox['pc'],Int(1)),#pc==1
             Equals(prox['r'],curr['r_']),
             Equals(prox['r_'],Minus(curr['r'],Times(curr['q'],curr['r_']))),
             Equals(prox['s'],curr['s_']),
             Equals(prox['s_'],Minus(curr['s'],Times(curr['q'],curr['s_']))),
             Equals(prox['t'],curr['t_']),
             Equals(prox['t_'],Minus(curr['t'],Times(curr['q'],curr['t_']))),
        )
    

  #  print(t31)
  #  print("\n")
             
    t14= And(Equals(curr['pc'],Int(1)),#pc==1
             Equals(curr['r_'],Int(0)),#r'==0
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(4))#pc==4
            )
    
  #  print(t14)
  #  print("\n")
    
    t44= And(Equals(curr['pc'],Int(4)),
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(curr['q'],prox['q']),
             Equals(prox['pc'],Int(4))
            )
  #  print(t44)
  #  print("\n")
    
    return Or(t01,t12,t14,t23,t31,t44)

In [5]:
def genTrace(vars,init,trans,n):
    with Solver(name="z3") as s:
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                str=" "
                for v in X[i]:
                    str=str+v+f" = {s.get_value(X[i][v])} "
                print("State:",i,str)

# Exemplos 

In [6]:
n=12
N=15

a=12
b=6

genTrace(["pc","r","r_","s","s_","t","t_","q"],init,trans,n)

State: 0  pc = 0 r = 12 r_ = 0 s = 1 s_ = 0 t = 0 t_ = 0 q = 0 
State: 1  pc = 1 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 
State: 2  pc = 2 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 
State: 3  pc = 3 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 2 
State: 4  pc = 1 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 5  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 6  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 7  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 8  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 9  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 10  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 
State: 11  pc = 4 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 


In [7]:
n=13
N=31

a=30
b=7

genTrace(["pc","r","r_","s","s_","t","t_","q"],init,trans,n)

State: 0  pc = 0 r = 30 r_ = 0 s = 1 s_ = 0 t = 0 t_ = 0 q = 0 
State: 1  pc = 1 r = 30 r_ = 7 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 
State: 2  pc = 2 r = 30 r_ = 7 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 
State: 3  pc = 3 r = 30 r_ = 7 s = 1 s_ = 0 t = 0 t_ = 1 q = 4 
State: 4  pc = 1 r = 7 r_ = 2 s = 0 s_ = 1 t = 1 t_ = -4 q = 0 
State: 5  pc = 2 r = 7 r_ = 2 s = 0 s_ = 1 t = 1 t_ = -4 q = 0 
State: 6  pc = 3 r = 7 r_ = 2 s = 0 s_ = 1 t = 1 t_ = -4 q = 3 
State: 7  pc = 1 r = 2 r_ = 1 s = 1 s_ = -3 t = -4 t_ = 13 q = 0 
State: 8  pc = 2 r = 2 r_ = 1 s = 1 s_ = -3 t = -4 t_ = 13 q = 0 
State: 9  pc = 3 r = 2 r_ = 1 s = 1 s_ = -3 t = -4 t_ = 13 q = 2 
State: 10  pc = 1 r = 1 r_ = 0 s = -3 s_ = 7 t = 13 t_ = -30 q = 0 
State: 11  pc = 4 r = 1 r_ = 0 s = -3 s_ = 7 t = 13 t_ = -30 q = 1 
State: 12  pc = 4 r = 1 r_ = 0 s = -3 s_ = 7 t = 13 t_ = -30 q = 1 


## Provar por k-indução a propriedade 
## `safety = (r > 0) and (r < N) and (r = a*s + b*t`)

Para a prova desta propriedade, criamos então uma função chamada **safety** que será então a representação em código desta invariante e será usada na função **kinduction_always** (usada nas aulas).

In [8]:
#safety = (r > 0) and (r < N) and (r = a*s + b*t)
def safety(state):
    #print(And(GT(state['r'],Int(0)),LT(state['r'],Int(N)), Equals(state['r'],Plus(Times(Int(a),state['s']),Times(Int(b),state['t'])))))
    return And(GT(state['r'],Int(0)),LT(state['r'],Int(N)), Equals(state['r'],Plus(Times(Int(a),state['s']),Times(Int(b),state['t']))))

Tendo em conta que já não é usada a função declare tivemos então de adaptar o codigo da função **kinduction_always** de forma a receber e usar a função **genState**.

Algo observado é que para valores inferiores a 5, o programa crasha, obtendo o erro SolverReturnedUnknownResultError.

In [9]:
def kinduction_always(var,genState, init, trans,inv,n):
    with Solver(name="z3") as solver:
        s = [genState(var, 'X', i) for i in range(n+1)]
        solver.add_assertion(init(s[0]))
        
        for i in range(n-1):
            solver.add_assertion(trans(s[i], s[i+1]))
        
    #    solver.add_assertion(Equals(s[-1]['pc'], Int(4)))
        
        for i in range(n):
            solver.push()
            solver.add_assertion(Not(inv(s[i])))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                for i, state in enumerate(s):
                    print(f" pc = {solver.get_value(state['pc'])},r = {solver.get_value(state['r'])}, a = {a}, s = {solver.get_value(state['s'])}, b = {b}, t = {solver.get_value(state['t'])}.")
                return
            solver.pop()
        
        s2 = [genState(var, 'X', i+n) for i in range(n+1)]
        
        for i in range(n):
            solver.add_assertion(inv(s2[i]))
            solver.add_assertion(trans(s2[i],s2[i+1]))
        
        solver.add_assertion(Not(inv(s2[-1])))
        
     #   solver.add_assertion(Equals(s2[-1]['pc'], Int(4)))
        
        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            for i, state in enumerate(s):
                print(f" pc = {solver.get_value(state['pc'])},r = {solver.get_value(state['r'])}, a = {a}, s = {solver.get_value(state['s'])}, b = {b}, t = {solver.get_value(state['t'])}.")
            return
        
        print(f"> A propriedade verifica-se por k-indução (k={n}).")
   

# Exemplos

In [10]:
N=14

a=12
b=6

kinduction_always(["pc","r","r_","s","s_","t","t_","q"],genState,init,trans,safety,4)  

SolverReturnedUnknownResultError: 

In [ ]:
N=8

a=5
b=5

kinduction_always(["pc","r","r_","s","s_","t","t_","q"],genState,init,trans,safety,23)  

## Provar usando o model checking

Para provar por model checking recorrendo ao uso de interpolantes usaremos então a função dada nas aulas **model_checking** que requere o uso do solver msat.
Este solver não suporta as operações de divisão e de multiplicação, sendo assim temos de construir uma nova função de transições a ,**trans1** que será uma adaptação da função **trans** usada nos exercicios anteriores.

Foram acrescentadas **5** novas variavéis, **6** novas transições e duas transições foram ligeiramente alteradas:

**div** = é incrementada a cada divisão até que o valor de **q** seja menor que **r'** e será o valor que **q** toma no final.

**somar** , **somas** , **somat** = estas guardarão o valor da soma de **r'**, **s'** e **t'**  **q** vezes, respetivamente.

**conta** = é incrementada a cada soma até atingir o valor de **q**.

Transição 23 :
(Foi ligeiramente alterada, esta será para os casos em que o numerador (r) é inferior ao denominador (r') ,dando o resultado da divisão 0)

$$(\mathit{pc} = 2 \wedge r < r' \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{nextpc} =3 \wedge nextq = 0 )$$

Transição 25 :
(Se o numerador(r) for maior ou igual ao denominador (r') então **q** tem a primeira subtração entre r e r' e div encontra-se a 1)

$$(\mathit{pc} = 2 \wedge r >= r' \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{nextpc} = 5 \wedge (nextq = r - r') \wedge nextdiv = 1 )$$

Transição 55:
(Enquanto o q for maior ou igual ao r' então são feitas subtrações cujo resultado é guardado em **q** e div é incrementado)

$$(\mathit{pc} = 5 \wedge q >= r' \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{nextpc} = 5 \wedge (nextq = q - r') \wedge (nextdiv = div + 1) )$$

Transição 53:
(No final das subtrações sucessivas **q** fica então com o valor de div)

$$(\mathit{pc} = 5 \wedge q < r' \wedge r = next r \wedge r' = next r' \wedge s = next s \wedge s' = next s' \wedge t = next t \wedge t' = next t' \wedge \mathit{nextpc} = 3 \wedge nextq = div \wedge nextdiv = div )$$

Transição 31:
(Foi ligeiramente alterada, esta será para os casos em que **q** dá 0 , tornando redudante a operação de multiplicação)

$$(\mathit{pc} = 3 \wedge q = 0 \wedge \mathit{nextpc} = 1 \wedge nextr = r' \wedge nextr' =r \wedge nexts = s' \wedge nexts' = s \wedge nextt= t' \wedge nextt' = t )$$

Transição 36:
(Se **q** for então diferente de 0, **somar** fica então com o valor de r', **somas** com o valor de s' , **somat** com o valor de t' e conta encontra-se a 1)

$$(\mathit{pc} = 3 \wedge q != 0 \wedge \mathit{nextpc} = 6 \wedge nextr = r \wedge nextr' =r' \wedge nexts = s \wedge nexts' = s' \wedge nextt= t \wedge nextt' = t' \wedge nextq = q \wedge nextconta = 1 \wedge nextsomar = r' \wedge nextsomas = s' \wedge nextsomat = t')$$

Transição 66:
(Enquanto conta for diferente do valor de **q** são **somar** incrementa r', **somas** incrementa s' , **somat** incrementa t' e conta é incrementado)

$$(\mathit{pc} = 6 \wedge conta != q \wedge \mathit{nextpc} = 6 \wedge nextr = r \wedge nextr' =r' \wedge nexts = s \wedge nexts' = s' \wedge nextt= t \wedge nextt' = t' \wedge nextq = q \wedge (nextconta = conta + 1) \wedge (nextsomar = somar + r') \wedge (nextsomas = somas + s') \wedge (nextsomat = somat + t'))$$

Transição 61:
(No final das somas sucessivas fazemos então os cálculos pretendidos , pois já temos os resultados das multiplicações)

$$(\mathit{pc} = 6 \wedge conta = q \wedge \mathit{nextpc} = 1 \wedge nextr = r' \wedge (nextr' = r - somar) \wedge nexts = s' \wedge (nexts' = s - somas) \wedge nextt= t' \wedge (nextt' = t - somat) )$$


In [11]:
def trans1(curr, prox):
    
    t01= And(
             Equals(curr['pc'],Int(0)),#pc==0
             Equals(prox['pc'],Int(1)),#pc==1 
             Equals(prox['r'],Int(a)),#r==a
             Equals(prox['r_'],Int(b)),#r'==b
             Equals(prox['s'],Int(1)),#s==1
             Equals(prox['s_'],Int(0)),#s'==0
             Equals(prox['t'],Int(0)),#t==0
             Equals(prox['t_'],Int(1))#t'==1
            )
    
  #  print(t01)
  #  print("\n")
    
    t12= And(Equals(curr['pc'],Int(1)),#pc==1
             Not(Equals(curr['r_'],Int(0))),#r'!=0
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(2))#pc==2
            )
    
  #  print(t12)
  #  print("\n")

    t25= And(Equals(curr['pc'],Int(2)),#pc==2
             GE(curr['r'],curr['r_']),#r>=r_
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(5)),#pc==5
             Equals(prox['q'],Minus(curr['r'],curr['r_'])),
             Equals(prox['div'],Int(1))
    )
    
  #  print(t25)
  #  print("\n")
    
    t55= And(Equals(curr['pc'],Int(5)),#pc==5
             GE(curr['q'],curr['r_']),#q>=r_
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(5)),#pc==5
             Equals(prox['q'],Minus(curr['q'],curr['r_'])),
             Equals(prox['div'],Plus(curr['div'],Int(1)))
    )
    
  #  print(t55)
  #  print("\n")
    
    t53= And(Equals(curr['pc'],Int(5)),#pc==5
             LT(curr['q'],curr['r_']),#q<r_
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(3)),#pc==3
             Equals(prox['q'],curr['div']),
             Equals(prox['div'],curr['div'])
    )
    
  #  print(t53)
  #  print("\n")
    
    t23= And(Equals(curr['pc'],Int(2)),#pc==2
             LT(curr['r'],curr['r_']),
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(3)),#pc==3
             Equals(prox['q'],Int(0))
    )

  #  print(t23)
  #  print("\n")
    
    t31= And(Equals(curr['pc'],Int(3)),#pc==3
             Equals(curr['q'],Int(0)),
             Equals(prox['pc'],Int(1)),#pc==1
             Equals(prox['r'],curr['r_']),
             Equals(prox['r_'],curr['r']),
             Equals(prox['s'],curr['s_']),
             Equals(prox['s_'],curr['s']),
             Equals(prox['t'],curr['t_']),
             Equals(prox['t_'],curr['t']),
        )
        
  #  print(t31)
  #  print("\n")
    
    t36= And(Equals(curr['pc'],Int(3)),#pc==3
             Not(Equals(curr['q'],Int(0))),
             Equals(prox['pc'],Int(6)),#pc==6
             Equals(prox['r'],curr['r']),
             Equals(prox['r_'],curr['r_']),
             Equals(prox['s'],curr['s']),
             Equals(prox['s_'],curr['s_']),
             Equals(prox['t'],curr['t']),
             Equals(prox['t_'],curr['t_']),
             Equals(prox['q'],curr['q']),
             Equals(prox['conta'],Int(1)),
             Equals(prox['somar'],curr['r_']),
             Equals(prox['somas'],curr['s_']),
             Equals(prox['somat'],curr['t_'])
        )
    
  #  print(t36)
  #  print("\n")
    
    t66= And(Equals(curr['pc'],Int(6)),#pc==6
             Not(Equals(curr['conta'],curr['q'])),
             Equals(prox['pc'],Int(6)),#pc==6
             Equals(prox['r'],curr['r']),
             Equals(prox['r_'],curr['r_']),
             Equals(prox['s'],curr['s']),
             Equals(prox['s_'],curr['s_']),
             Equals(prox['t'],curr['t']),
             Equals(prox['t_'],curr['t_']),
             Equals(prox['q'],curr['q']),
             Equals(prox['conta'],Plus(curr['conta'],Int(1))),
             Equals(prox['somar'],Plus(curr['somar'],curr['r_'])),
             Equals(prox['somas'],Plus(curr['somas'],curr['s_'])),
             Equals(prox['somat'],Plus(curr['somat'],curr['t_'])),
        )
    
  #  print(t66)
  #  print("\n")
    
    t61= And(Equals(curr['pc'],Int(6)),#pc==6
             Equals(curr['conta'],curr['q']),
             Equals(prox['pc'],Int(1)),#pc==1
             Equals(prox['r'],curr['r_']),
             Equals(prox['r_'],Minus(curr['r'],curr['somar'])),
             Equals(prox['s'],curr['s_']),
             Equals(prox['s_'],Minus(curr['s'],curr['somas'])),
             Equals(prox['t'],curr['t_']),
             Equals(prox['t_'],Minus(curr['t'],curr['somat'])),
        )
    

  #  print(t61)
  #  print("\n")
             
    t14= And(Equals(curr['pc'],Int(1)),#pc==1
             Equals(curr['r_'],Int(0)),#r'==0
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(prox['pc'],Int(4))#pc==4
            )
    
  #  print(t14)
  #  print("\n")
    
    t44= And(Equals(curr['pc'],Int(4)),
             Equals(curr['r'],prox['r']),
             Equals(curr['r_'],prox['r_']),
             Equals(curr['s'],prox['s']),
             Equals(curr['s_'],prox['s_']),
             Equals(curr['t'],prox['t']),
             Equals(curr['t_'],prox['t_']),
             Equals(curr['q'],prox['q']),
             Equals(prox['pc'],Int(4))
            )
  #  print(t44)
  #  print("\n")
    
    return Or(t01,t12,t14,t23,t25,t55,t53,t31,t36,t66,t61,t44)

# Exemplo

Como podemos ver, são necessários mais passos para determinar o gdc.

In [12]:
n=13
N=13

a=12
b=6

#gera_traco(k)
genTrace(["pc","r","r_","s","s_","t","t_","q","div","somar","somas","somat","conta"],init,trans1,n)

State: 0  pc = 0 r = 12 r_ = 0 s = 1 s_ = 0 t = 0 t_ = 0 q = 0 div = 0 somar = 0 somas = 0 somat = 0 conta = 0 
State: 1  pc = 1 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 div = 0 somar = 0 somas = 0 somat = 0 conta = 0 
State: 2  pc = 2 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 div = 0 somar = 0 somas = 0 somat = 0 conta = 0 
State: 3  pc = 5 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 6 div = 1 somar = 0 somas = 0 somat = 0 conta = 0 
State: 4  pc = 5 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 0 div = 2 somar = 0 somas = 0 somat = 0 conta = 0 
State: 5  pc = 3 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 2 div = 2 somar = 0 somas = 0 somat = 0 conta = 0 
State: 6  pc = 6 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 2 div = 0 somar = 6 somas = 0 somat = 1 conta = 1 
State: 7  pc = 6 r = 12 r_ = 6 s = 1 s_ = 0 t = 0 t_ = 1 q = 2 div = 0 somar = 12 somas = 0 somat = 2 conta = 2 
State: 8  pc = 1 r = 6 r_ = 0 s = 0 s_ = 1 t = 1 t_ = -2 q = 0 div = 0 somar = 0 somas = 0 somat = 0 co

Por fim , criaremos a função **error** que será a negação da propriedade safety e usaremos a função **model_checking** para provar.

# Exemplos

In [13]:
def error(state):
    #print(And(GT(state['r'],Int(0)),LT(state['r'],Int(N)), Equals(state['r'],Plus(Times(Int(a),state['s']),Times(Int(b),state['t'])))))
    return Not(And(GT(state['r'],Int(0)),LT(state['r'],Int(N)), Equals(state['r'],Plus(Times(Int(a),state['s']),Times(Int(b),state['t'])))))

def invert(trans):
    return lambda curr, prox: trans(prox, curr)

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def model_checking(vars,init,trans,error,N,M):
    with Solver(name="msat") as solver:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        transt = invert(trans)
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        # Step 1 implícito na ordem de 'order' e nas definições de Rn, Um.
        for (n,m) in order:      
            # Step 2. 
            I = init(X[0])
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Rn = And(I, Tn)
            
            E = error(Y[0])
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro.")
                return
            else:
                # Step 3. 
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)
                
                # Salvaguardar cálculo bem-sucedido do interpolante.
                if C is None:
                    print("> O interpolante é None.")
                    break
                
                # Step 4. 
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    # C é invariante de T.
                    print("> O sistema é seguro.")
                    return 
                else:
                    # Step 5.1.
                    S = rename(C, X[n])
                    while True:
                        # Step 5.2.
                        T = trans(X[n], Y[m])
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante.")
                            break 
                        else:
                            # Step 5.3.
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                # Step 5.4.
                                # C(Xn) -> S é tautologia.
                                print("> O sistema é seguro.")
                                return
                            else:
                                # Step 5.5.
                                # C(Xn) -> S não é tautologia.
                                S = Or(S, Cn)
                                
    print("> Não foi provada a segurança ou insegurança do sistema.")          

In [14]:
N=13

a=12
b=6

model_checking(['pc','a','b','r','r_','s','s_','t','t_','q','div','conta','somar','somas','somat'], init, trans1, error, 50, 50)

> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> Não foi encontrado majorante.
> O sistema é seguro.


In [15]:
N=8

a=12
b=6

model_checking(['pc','a','b','r','r_','s','s_','t','t_','q','div','conta','somar','somas','somat'], init, trans1, error, 50, 50)

> O sistema é seguro.


Dá logo seguro pois neste exemplo **a** é maior que N, o que contrai a pré condição